In [15]:
import pandas as pd

# Specify the input and output CSV file paths
input_csv_file = '../data/train_data/edges.csv'  # Replace with your input file path
output_csv_file = '../data/small_100/edges.csv'  # Replace with your desired output file path

# Read the first 100 lines of the CSV file
data = pd.read_csv(input_csv_file, nrows=100)

# Save the first 100 lines to a new CSV file
data.to_csv(output_csv_file, index=False)

In [18]:
import _path
from txgnn import TxData, TxGNN, TxEval

Tx_data = TxData(data_folder_path = '../data/train_data')
Tx_data.prepare_split(split = 'complex_disease', seed = 42, no_kg = False)

TxGNN = TxGNN(data = Tx_data, 
          weight_bias_track = False,
          proj_name = 'TxGNN',
          exp_name = 'TxGNN',
          #device='cpu'
          )


Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...


Splits detected... Loading splits....
Creating DGL graph....
Done!


In [23]:
#TxGNN.load_pretrained(f'./models_{split_name}_{seed_no}/')
import glob
models = glob.glob('../models/checkpoints_all_seeds/*')
for m in models:
    print(m)
    try:
        TxGNN.load_pretrained('../models/checkpoints_all_seeds/TxGNN_4_random')

        # Get model architecture
        print(TxGNN.graphmask_model)

        # Get model parameters
        for name, param in TxGNN.model.named_parameters():
            print(f'Parameter: {name}, Shape: {param.data.shape}')
    except AttributeError:
        pass

../models/checkpoints_all_seeds/TxGNN_5_cell_proliferation
../models/checkpoints_all_seeds/TxGNN_1_anemia
../models/checkpoints_all_seeds/TxGNN_3_complex_disease
../models/checkpoints_all_seeds/TxGNN_3_adrenal_gland
../models/checkpoints_all_seeds/TxGNN_5_anemia
../models/checkpoints_all_seeds/TxGNN_5_metabolic_disorder
../models/checkpoints_all_seeds/TxGNN_2_neurodigenerative
../models/checkpoints_all_seeds/TxGNN_2_anemia
../models/checkpoints_all_seeds/TxGNN_2_cardiovascular
../models/checkpoints_all_seeds/TxGNN_1_complex_disease
../models/checkpoints_all_seeds/TxGNN_4_anemia
../models/checkpoints_all_seeds/TxGNN_4_cardiovascular
../models/checkpoints_all_seeds/TxGNN_4_mental_health
../models/checkpoints_all_seeds/TxGNN_4_neurodigenerative
../models/checkpoints_all_seeds/TxGNN_2_metabolic_disorder
../models/checkpoints_all_seeds/TxGNN_1_cardiovascular
../models/checkpoints_all_seeds/TxGNN_3_autoimmune
../models/checkpoints_all_seeds/TxGNN_3_anemia
../models/checkpoints_all_seeds/TxGN

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_params = count_parameters(TxGNN.model)
print(f'Number of parameters in the model: {num_params}')

Number of parameters in the model: 1015000


In [4]:

num_nodes = TxGNN.G.number_of_nodes()
print(f'Number of nodes: {num_nodes}')
num_edges = TxGNN.G.number_of_edges()
print(f'Number of edges: {num_edges}')

Number of nodes: 129312
Number of edges: 6735722


In [5]:
TxGNN.train_graphmask(relation = 'indication',
              learning_rate = 3e-4,
              allowance = 0.005,
              epochs_per_layer = 0,
              penalty_scaling = 1,
              valid_per_n = 20)

gate_hidden_size:  32
Enabling layer 1
Enabling layer 0


UnboundLocalError: local variable 'epoch' referenced before assignment

In [7]:
GM_num_params = count_parameters(TxGNN.graphmask_model)
print(f'Number of parameters in the model: {GM_num_params}')

Number of parameters in the model: 669300


In [8]:
batch_size = 1
features_size = 1
memory_for_params = (GM_num_params * 4) / (1024 * 1024)  # Memory in MB

# Calculate memory for activations (simplified estimate)
memory_for_activations = batch_size * (num_nodes + num_edges) * features_size * 4 / (1024 * 1024)  # Memory in MB

# Total memory requirement
total_memory = memory_for_params + memory_for_activations
print(f'Total memory requirement: {total_memory:.2f} MB')

Total memory requirement: 28.74 MB
